Instructions to run the `finetune_runner.py` script to generate additional fine-tuning results for each of the 3 weights scenarios:

1. `random`
2. `10` (10% pre-train weights)
3. `20` (20% pre-train weights)

Data refers to the percentage of patients in the _Icentia11K_ dataset.

## Instructions

Prerequisites:

- Google Colab Pro
- V100 GPU


### Select instance

Select V100 GPU runtime.

### Clone Git repo

In [ ]:
%cd /root
! git clone https://github.com/myles-i/DLH_TransferLearning.git
%cd DLH_TransferLearning

### Install dependencies



In [ ]:
%%capture
! pip install -r requirements.txt

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Determine path to data and job directories on Drive

Example for me:

`/content/drive/MyDrive/DLHProject`

where `DLHProject` is the name of my shortcut for the shared folder.

In [ ]:
PROJECT_DIR = '/content/drive/MyDrive/DLHProject'
DATA_DIR = PROJECT_DIR + '/data'
JOB_DIR = PROJECT_DIR + '/jobs'

In [ ]:
# setup top level directory paths
PROJECT_DIR = '/content/drive/MyDrive/DLHProject'
DATA_DIR = PROJECT_DIR + '/data'
JOB_DIR = PROJECT_DIR + '/jobs'

In [ ]:
import os
dry_run = True # change this to False once you have verified everything looks good

# finetune/pretraining data paths (can vary depending on model used)
arch = "resnet18_2d"
PHYSIONET_TRAIN = DATA_DIR + "/physionet_finetune_spectrogram/physionet_train.pkl"
PHYSIONET_TEST = DATA_DIR + "/physionet_finetune_spectrogram/physionet_test.pkl"
PRETRAIN_BASE =JOB_DIR + "/spectrogram/pretraining/min_normalization_16epochs_to_20percent"
FINETUNE_BASE =JOB_DIR + "/spectrogram/finetuning/min_normalization_16epochs_to_20percent"
os.makedirs(FINETUNE_BASE, exist_ok=True)

# define which combinations of runs to do
weight_types = {"random",
                "10",
                "20"}

weight_files = {"",
                PRETRAIN_BASE + "/epoch_08/model.weights",
                PRETRAIN_BASE + "/epoch_16/model.weights"}


os.makedirs(FINETUNE_BASE, exist_ok=True)
for weight_type, weight_file in zip(weight_types, weight_files):
  # lets verify all data/weight files exist
  assert os.path.exists(PHYSIONET_TRAIN), "PHYSIONET_TRAIN does not exist"
  assert os.path.exists(PHYSIONET_TEST), "PHYSIONET_TEST does not exist"
  assert os.path.exists(weight_file), "weight_file does not exist"

  ! seq 10 10 100 | xargs -P 1 -I {} python finetune_runner.py \
  --arch {arch} \
  --weights-type {weight_type} \
  --weights-file {weight_file} \
  --job-base-dir /content/drive/MyDrive/DLHProject/jobs/spectrogram \
  --train {PHYSIONET_TRAIN} \
  --test {PHYSIONET_TEST} \
  --batch-size 128 \
  --epochs 200 \
  --seed '{}' \
  --dryrun {dry_run}